# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [15]:
####################### TEST NAME ###################################
test = 'Esophagus'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_path = '/home/rstudio/data/anndata/Esophagus.h5ad'
#Manually set existing QC Columns
mean_insertsize = None
insertsize_count = None
n_total_fragments = None
n_fragments_in_promoters = None
pct_fragments_in_promoters = None
blacklist_overlaps = None
#total_number_of_fragments
TN = 'TN'
#uniquely_mapped_fragments
UM = 'UM'
#properly_paired_fragments
PP = 'PP'
#uniq_fragments
UQ = 'UQ'
#chrM_fragments
CM = 'CM'

#################### Calc QC Columns ###############################
#set available number of threads
n_threads = 8

#if the mean_insertsize should be calculated specify either a BAM file or a fragments file:
fragments_file = '/home/rstudio/data/bamfiles/fragments_Esophagus.bed'
bam_file = '/home/rstudio/data/bamfiles/sorted_Esophagus.bam'

#specify promotors_gtf if another promoter than the fixed ones should be used
#if promotors is None please specify species
#promoters_gtf =  '/mnt/flatfiles/organisms/new_organism/homo_sapiens/104/homo_sapiens.104.promoters2000.gtf'
promoters_gtf = '/home/rstudio/data/homo_sapiens.104.promoters2000.gtf'
species = None

##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [2]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
from sctoolbox.qc_filter import *
from sctoolbox.atac_utils import *
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [3]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

/home/rstudio/processed_data/Esophagus/qc/anndata: NEWLY SETUP
all directories existing
all directories existing


## Read in data

### Option 1: Read from .h5ad

In [4]:
qc_columns = {}
qc_columns["mean_insertsize"] = mean_insertsize
qc_columns['n_total_fragments'] = n_total_fragments
qc_columns['n_fragments_in_promoters'] = n_fragments_in_promoters
qc_columns['pct_fragments_in_promoters'] = pct_fragments_in_promoters
qc_columns["blacklist_overlaps"] = blacklist_overlaps
qc_columns["TN"] = TN
qc_columns["UM"] = UM
qc_columns["PP"] = PP
qc_columns["UQ"] = UQ
qc_columns["CM"] = CM

In [5]:
if from_h5ad:
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")
    
    print('add existing adata.obs columns to infoprocess:')
    print()
    for key, value in qc_columns.items():
        if value is not None:
            print(key + ':' + value)
            if value in adata.obs.columns:
                build_legend(adata, key, value)
            else:
                print('column:  ' + value + ' is not in adata.obs')
    
    if not adata.obs.index.name == "barcode":
        print('setting adata.obs.index = adata.obs[barcode]')
        adata.obs = adata.obs.set_index("barcode")
    else:
        print('barcodes are already the index')
    

add existing adata.obs columns to infoprocess:

TN:TN
UM:UM
PP:PP
UQ:UQ
CM:CM
setting adata.obs.index = adata.obs[barcode]


## Inspect adata

In [6]:
display(adata)

AnnData object with n_obs × n_vars = 73652 × 150152
    obs: 'TN', 'UM', 'PP', 'UQ', 'CM', 'file', 'sample'
    var: 'name'
    uns: 'infoprocess', 'color_set', 'legend'

In [7]:
display(adata.var)

,name
0,b'chr1':10008-10690
1,b'chr1':11179-11381
2,b'chr1':28734-29439
3,b'chr1':29495-29682
4,b'chr1':34673-35029
...,...
150147,b'chrY':57203305-57203488
150148,b'chrY':57206427-57206551
150149,b'chrY':57215321-57215411
150150,b'chrY':57215521-57215837


In [8]:
display(adata.obs)

,TN,UM,PP,UQ,CM,file,sample
barcode,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/Esophagus.snap,MB
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/Esophagus.snap,MB
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA,9.0,6.0,6.0,6.0,0.0,/opt/project/data/snap/Esophagus.snap,MB
TTCCATCCTCTTTCTGCAGACT,10.0,6.0,6.0,6.0,1.0,/opt/project/data/snap/Esophagus.snap,MB
TTCCATCCTCTTTCTGGCGCAG,17.0,5.0,5.0,5.0,1.0,/opt/project/data/snap/Esophagus.snap,MB


## Calculate missing columns

### 1. calc mean insertsize and count if missing 

In [11]:
if qc_columns['mean_insertsize'] is None or qc_columns['insertsize_count'] is None:
    atac.add_insertsize(adata, fragments=fragments_file) # or use bam=bam_file instead of fragments
    
    # update column dictionary
    qc_columns['mean_insertsize'] = 'mean_insertsize'
    qc_columns['insertsize_count'] = 'insertsize_count'
    
    # update infoprocess table
    build_legend(adata, 'mean_insertsize', 'mean_insertsize')
    build_legend(adata, 'insertsize_count', 'insertsize_count')
    
adata.obs

Counting fragment lengths from fragments file...
Done reading file - elapsed time: 0:03:54
Converting counts to dataframe...
Done getting insertsizes from fragments!
Added insertsize information to adata.obs[["insertsize_count", "mean_insertsize"]] and adata.uns["insertsize_distribution"].


,TN,UM,PP,UQ,CM,file,sample,insertsize_count,mean_insertsize
barcode,,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,12.0,85.18
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,26.0,167.83
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,12.0,152.17
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/Esophagus.snap,MB,5066.0,135.61
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,23.0,126.76
...,...,...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA,9.0,6.0,6.0,6.0,0.0,/opt/project/data/snap/Esophagus.snap,MB,6.0,117.17
TTCCATCCTCTTTCTGCAGACT,10.0,6.0,6.0,6.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,7.0,54.43
TTCCATCCTCTTTCTGGCGCAG,17.0,5.0,5.0,5.0,1.0,/opt/project/data/snap/Esophagus.snap,MB,5.0,73.80


In [29]:
#adata.uns['legend']

### 2. promotor enrichment 

In [17]:
if qc_columns['n_fragments_in_promoters'] is None:
    overlap.pct_fragments_in_promoters(adata, gtf_file=promoters_gtf, species=species, bam_file=bam_file, cb_col=None, nproc=1)
    
    # n_total_fragments, n_fragments_in_promoters, pct_fragments_in_promoters
    # update column dictionary
    qc_columns['n_total_fragments'] = 'n_total_fragments'
    qc_columns['n_fragments_in_promoters'] = 'n_fragments_in_promoters'
    qc_columns['pct_fragments_in_promoters'] = 'pct_fragments_in_promoters'
    # update infoprocess table
    build_legend(adata, 'n_total_fragments', 'n_total_fragments')
    build_legend(adata, 'n_fragments_in_promoters', 'n_fragments_in_promoters')
    build_legend(adata, 'pct_fragments_in_promoters', 'pct_fragments_in_promoters')
    

Converting GTF to BED...
Converting BAM to fragments file! This may take a while...
Finished creating fragments file. Now sorting...
Finished sorting fragments
Finding overlaps...


FileNotFoundError: File "/home/rstudio/data/bamfiles/sorted_Esophagus_fragments_sorted_homo_sapiens.104.promoters2000.gtf_sorted_overlap.bed" does not exist

## Inspect adata.obs

In [31]:
adata.obs

,TN,UM,PP,UQ,CM,file,sample,insertsize_count,mean_insertsize,n_total_fragments,n_fragments_in_promoters,pct_fragments_in_promoters
barcode,,,,,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,44,90.91,15,5.0,0.333333
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,82,143.39,35,8.0,0.228571
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,38,151.21,18,7.0,0.388889
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/cropped_146.snap,MB,11860,130.28,5118,1223.0,0.238961
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,60,113.80,28,6.0,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...
AAACTACCAGCTTGGCAACAGC,26.0,20.0,20.0,20.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,44,102.77,19,9.0,0.473684
AAACTACCAGCTTTAACTGCGC,29.0,20.0,20.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,42,124.00,19,5.0,0.263158
AAACTACCAGCTTTAGCCCTAT,32.0,24.0,24.0,24.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,54,117.11,21,4.0,0.190476


## Save adata to .h5ad

In [32]:
adata_output = tree.assembled_anndata
adata_output

'/home/rstudio/processed_data/cropped_146/assembling/anndata/cropped_146.h5ad'

In [33]:
import numpy as np
import pandas as pd

In [34]:
adata.uns['insertsize_distribution'] = adata.uns['insertsize_distribution'].swapaxes('index', 'columns')

In [35]:
adata.uns['insertsize_distribution'] = adata.uns['insertsize_distribution'].reset_index(drop=True)

In [36]:
adata.uns['insertsize_distribution']

,AAACTACCAGAAACCCGAGATA,AAACTACCAGAAACCTAAGTGG,AAACTACCAGAAACGGATCAGT,AAACTACCAGAAACGTCCCGTT,AAACTACCAGAAACTAGCCCTA,AAACTACCAGAAAGCACTAGCG,AAACTACCAGAAAGCCCACGAC,AAACTACCAGAAAGCGGGAGCT,AAACTACCAGAAAGGAACAGAC,AAACTACCAGAAAGGATCGGCT,...,AAACTACCAGCTTGAGACTCCG,AAACTACCAGCTTGCGGGTGTT,AAACTACCAGCTTGCGTATCAG,AAACTACCAGCTTGCTGCCTAT,AAACTACCAGCTTGGATGATGC,AAACTACCAGCTTGGCAACAGC,AAACTACCAGCTTTAACTGCGC,AAACTACCAGCTTTAGCCCTAT,AAACTACCAGCTTTAGCTGACT,AAACTACCAGCTTTGGGACGGT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)